# Tutorial: Entendimiendo de los datos

Continuando con el proyecto de consultoria de Wide World Importers, el primer paso para iniciar la comprensión de los datos es explorar y entender las fuentes de datos disponibles. Note que esto también nos ayuda a comprender mejor la organización.

## Configuración e importe de paquetes
Se utilizará el paquete de pandas profiling para apoyar el análisis estadístico, y se importan los paquetes de python
necesarios

In [1]:
import pyspark
from pyspark.sql import SparkSession 
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
import pyspark.sql.functions as f
import os 
from datetime import datetime
from pyspark.sql import types as t
from pandas_profiling import ProfileReport
#import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'pyspark'

Configuración del controlador e inicio de sesion Spark

In [ ]:
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [ ]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

In [3]:
# Si quiere practicar la conexion con el servidor de base de datos:
db_connection_string = 'jdbc:mysql://157.253.236.116:8080/WWImportersTransactional'
# El usuario es su estudiante _i asignado y su contraseña la encontrará en el archivo excel de Coursera 
db_user = ''
db_psswd = ''

PATH='./'

### Información dada por la organización relacionada con los datos

En esta actividad le recomendamos revisar el diccionario de datos suministrado y en particular la información relacionada con esta tabla. Si tienen alguna duda relacionada a esta tabla deben incluirla en las preguntas o aclaraciones que le entreguen al negocio. En relación a los datos Wide World Importers menciona:

- La cantidad máxima de productos movidos es 50 millones por viaje
- Existen 236.668 movimientos de producto realizados desde 2013
- Los movimientos están relacionados con 228.265 clientes
- El formato de fechas manejado es YYYY-MM-DD HH:MM:SS
- Actualmente tenemos 13 proveedores

Idealmente debe terminar con una lista de preguntas o aclaraciones para la organización, conclusiones de los datos/problemas de calidad y alternativas para la limpieza.

### Conexión a fuente de datos y acceso a los datos

#### Conexión a fuente de datos
A continuación encuentra las funciones para conectarse a la fuente de datos (archivo csv o base de datos) y retornar un dataframe que es el que se utilizará posteriormente para manipular los datos.

In [ ]:
def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

#### Cargue de datos
Los criterios generales para la evaluación de la actividad son:
1. Completitud del perfilamiento de datos (40 puntos)
- Cantidad de registros y cantidad de columnas por tipo de dato 
- Significado de una fila 
- Estadísticas descriptivas por columna (numérica, categórica, fechas) 


In [ ]:
sql_movimientoscopia = '''(SELECT mc.TransaccionProductoID,
mc.ProductoID, NombreProducto,
tt.TipoTransaccionID, tt.TipoTransaccionNombre, 
mc.ClienteID,
mc.InvoiceID,
mc.ProveedorID, pc.NombreProveedor,
mc.OrdenDeCompraID,
mc.FechaTransaccion,
mc.Cantidad 
FROM movimientosCopia mc
INNER JOIN TiposTransaccion tt ON tt.TipoTransaccionID=mc.TipoTransaccionID
INNER JOIN Producto pr ON pr.ID_Producto=mc.ProductoID 
INNER JOIN provedoresCopia pc ON pc.ProveedorID=mc.ProveedorID
order by Cantidad desc) AS Compatible'''

Para ver la cantidad de registros, se utilizo el comando <i>Count</i> ingresando el identificador de la tabla TransaccionProductoID, de la tabla a consultar.

In [ ]:
sql_cantidadregistros = '''(SELECT  COUNT(mc.TransaccionProductoID) AS CATIDAD
from movimientosCopia mc ) AS CATIDAD'''

In [ ]:
#Se cargan los dataframes desde la base de datos
movimientoscopia  = obtener_dataframe_de_bd(db_connection_string, sql_movimientoscopia, db_user, db_psswd)
cantidadregistros = obtener_dataframe_de_bd(db_connection_string, sql_cantidadregistros, db_user, db_psswd)

In [ ]:
print(movimientoscopia.count())

### Entendimiento general de datos

<i>Significado de una fila </i>.

Como puede observar en la tabla de movimientosCopia se tiene informacion **relacionada con el producto, el tipo de transaccion, el proveedor, el cliente, el identificador de la factura, con la órden, fecha del movimiento en inventario y la cantidad de productos que se retiraron/ingresaron a inventario**. 

- La información del producto su identificador <i>ProductoID</i>, el cual representa el producto que fue extraido o agregado en el inventario.
- La información del tipo de transaccion <i>TipoTransaccionID</i>, el cual representa el tipo de transaccion del movimiento.
- La información del Cliente <i>ClienteID</i>, representa el cliente que hace la compra.
- La información de la factura <i>InvoiceID</i>, representa el Identificador de la factura.
- La información del Proveedor <i>ProveedorID</i>, representa el proveedor del producto en movimiento.
- La información de la orden <i>OrdenDeCompraID</i>, representa el id de la orden de este movimiento.
- La información de la fecha del movimiento <i>FechaTransaccion</i>, representa la fecha del movimiento en inventario.
- La información de la cantidad de productos <i>Cantidad</i>, representa la cantidad de productos que se retiraron/ingresaron a inventario.


<i>Estadísticas descriptivas por columna (numérica, categórica, fechas)</i> ingresando el número de filas que se quiere ver

In [5]:
movimientoscopia.schema

NameError: name 'ordenes' is not defined

In [1]:
El esquema del DataFrame incluye los nombres de las columnas asi como el tipo de dato. 
Recuerde que tipo de dato (e.g., booleano, caracteres, enteros, flotantes, colecciones) es diferente a tipo de variable 
(numérica y categórica).

SyntaxError: invalid syntax (2894725880.py, line 1)

## Análisis de calidad de datos

###  Completitud: Datos que no existen o no se conocen

Para ver los registros que no cumplen con el formato podemos negar la condición del filtro con el símbolo ~ al principio:

In [1]:
noCumplenFormato = movimientoscopia.movimientoscopia(~ordenes["FechaTransaccion"].rlike(regex))
noCumplenFormato.count(), noCumplenFormato.select(col("FechaTransaccion")).show()

NameError: name 'movimientoscopia' is not defined

Concluimos entonces, que hay 20.035 registros cuya fecha no cumple con el formato especificado. Sin embargo, los valores que son diferentes parecen serlo solo porque varia el formato, este tipo de casos pueden ser solucionados fácilmente por lo que una notificación al negocio sería con caracter informativo más que para solicitar una aclaración de su parte.

Para facilitar los análisis y evitar inconsistencias o conclusiones erradas vamos a estandarizar los datos, en este caso las fechas. Sin embargo, esta actividad puede hacerse sobre formatos de números, texto entre otras. 

### Completitud y validez

In [ ]:
def contar_vacios(df):
    resultados = []
    for c in df.columns:
        vacios = df.filter(df[c].isNull()).count()
        if vacios!=0:
            print('número de vacíos para columna '+c+': '+str( vacios ))
            resultados.append(vacios)
    return resultados

def cardinalidad(df):
    resultado = {}
    for c in df.columns:
        cardinalidad = df.select(col(c)).distinct().count()
        if cardinalidad>=df.count()*0.5:
            resultado[c] = cardinalidad
    return resultado

In [ ]:
print('detalles movimientoscopia: ')
columnas_vacias_movimientoscopia = contar_vacios(movimientoscopia)

In [ ]:
columnas_alta_cardinalidad_movimientoscopia = cardinalidad(movimientoscopia) 

### Consistencia
A nivel de consistencia algunos ejemplos de revisión son precios negativos, productos que aparecen en las ordenes pero que no esten registrados en la tabla de produtos. En nuestro caso vamos a verificar que todas las ordenes tengan detalle de orden y viceversa

In [ ]:
sql_consistencia = '''select mc.TransaccionProductoID, 
mc.ClienteID, cli.Nombre
from movimientosCopia mc
INNER JOIN TiposTransaccion tt ON tt.TipoTransaccionID=mc.TipoTransaccionID
INNER JOIN Clientes cli ON cli.ID_Cliente=mc.ClienteID'''

In [ ]:
#Se cargan los dataframes desde la base de datos
consistencia  = obtener_dataframe_de_bd(sql_consistencia, sql_movimientoscopia, db_user, db_psswd) 

Como se puede observar hay una diferencia de la cantidd de registros que no tienen detalles se pueden observar en la consulta, ya que estos en la mayoria asocial el ID_cliente con un valor 0 y en clietes con ClienteID, no existe.

## Herramientas para facilitar el entendimiento de datos

Algunas de las estadísticas antes desarrolladas pueden obtenerse por medio de herramientas como Pandas Profiling una herramienta muy útil, <b><i>Pero no perfecta ni completa</i></b>, es Pandas profiling que dado un conjunto de datos, nos muestra histogramas, estadísticas etc. A continuación guardamos el reporte en html

In [ ]:
reporte = ProfileReport(movimientoscopia.toPandas())
reporte.to_file(PATH+"reporte_prefilamiento.html")

## Conclusiones/resultados


**Conclusiones generales:**

1. En la tabla de movimientosCopia tenemos información **relacionada con el producto, el tipo de transaccion, el proveedor, el cliente, el identificador de la factura, con la órden, fecha del movimiento en inventario y la cantidad de productos que se retiraron/ingresaron a inventario**. Por otra parte, en los detalles de ordenes se tiene **información del pedido y del producto**
3. ¿Por que en el campo ClienteID de la tabla movimientosCopia, existen registros con valor 0, siendo que el la tabla Clientes no existe ningun registro con esta identificacion.?

**Conclusiones de calidad:**

1. Las columnas ClienteID, ProveedorID, tienen un valor igual a 0, o null es decir son columnas vacías por lo que se pueden eliminar.

No se por que no se pudo realizar el analisis completo con la erramienta, esta siempre me generaba un error que no permitia correro el ejercicio.